In [ ]:
pip install -e src/gym-idsgame

In [19]:
!pip uninstall gymnasium
!pip install gym==0.21.0
!pip install scikit-learn

!pip install gym-idsgame==1.0.12

  Using cached gym-0.21.0.tar.gz (1.5 MB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [3 lines of output]
      /opt/homebrew/anaconda3/envs/gym-idsgame/lib/python3.13/site-packages/setuptools/_distutils/dist.py:270: UserWarning: Unknown distribution option: 'tests_require'
        warnings.warn(msg)
      error in gym setup command: 'extras_require' must be a dictionary whose values are strings or lists of strings containing valid project/version requirement specifiers.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
  Using cached gym_idsgame-1.0.12-py3-none-any.whl.metadata (1.1 kB)
  Using cached

In [60]:
import gymnasium
from gymnasium.core import Env

def patched_reset(self):
    return self.reset()

Env.reset = patched_reset


In [61]:
import os
import gym
import gym_idsgame
import numpy as np
import torch
# src/environment/idsgame_wrapper.py
# from src.agents.dqn_agent import DQNAgent
from src.environment.compatibility_wrapper import GymCompatibilityWrapper
from src.utils.utils import print_summary
from src.utils.plotting import plot_results
# from src.utils import create_artefact_dirs

print('DONE IMPORTING')

DONE IMPORTING


In [73]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np

from src.environment.explorer import IDSGameExplorer
explorer = IDSGameExplorer()
# explorer.run_comprehensive_exploration()

In [74]:
import src.agents.ddqn_agent
print(src.agents.ddqn_agent.__file__)

/Users/krishnaasrinivas/Desktop/AI-Agent-for-Cyber-Security/src/agents/ddqn_agent.py


In [87]:
import os
import gymnasium as gym
import warnings
from datetime import datetime

# from gym_idsgame.agents.training_agents.q_learning.q_agent_config import QAgentConfig
# from gym_idsgame.agents.training_agents.q_learning.dqn.dqn_config import DQNConfig
from experiments.util import util
from experiments.util.plotting_util import read_and_plot_results
# from src.agents.ddqn_agent import DDQNAgent
from src.utils.utils import get_output_dir, print_summary
from src.environment.compatibility_wrapper import GymCompatibilityWrapper
from src.utils.plotting import plot_results

warnings.filterwarnings('ignore')
print('done')

done


In [88]:
env_name = "idsgame-random_attack-v8"
output_dir = os.getcwd()
random_seed = 42
env = gym.make(env_name, save_dir=output_dir + "results/data/" + str(random_seed))
# env = GymCompatibilityWrapper(env)

env = GymCompatibilityWrapper(env)
env = env.unwrapped

print("\nEnvironment Information:")
print(f"Observation Space: {env.observation_space}")
print(f"Action Space: {env.action_space}")


Environment Information:
Observation Space: Box(0, 9, (1, 11), int32)
Action Space: Discrete(30)


In [89]:
from src.agents.ddqn_agent import DDQNConfig, DDQNAgent, create_ddqn_agent
import numpy as np
import torch
import matplotlib.pyplot as plt
import gym
import time
import os
import datetime

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [99]:
# Set random seed for reproducibility
random_seed = 42
np.random.seed(random_seed)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Create output directory for results
current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
result_dir = f"./results/ddqn_pytorch_{current_time}"
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

# # Create environment
# env_name = "idsgame-random_defense-v0"
# env = gym.make(env_name, save_dir=result_dir)

# Print environment information
print(f"Environment: {env_name}")
print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Environment: idsgame-random_attack-v8
Action space: Discrete(30)
Observation space: Box(0, 9, (1, 11), int32)


In [102]:

from src.agents.ddqn_agent import DDQNConfig, DDQNAgent, create_ddqn_agent

# Configure DDQN agent\n",
ddqn_config = DDQNConfig(
       gamma=0.99,                       # Discount factor\n",
        lr=0.00005,                       # Learning rate\n",
        batch_size=64,                    # Batch size for training\n",
      epsilon=1.0,                      # Initial exploration rate\n",
       epsilon_decay=0.999,              # Decay rate for epsilon\n",
 min_epsilon=0.01,                 # Minimum value for epsilon\n",
   target_network_update_freq=100,   # Frequency of target network updates\n",
    replay_memory_size=10000,         # Size of replay buffer\n",
   num_episodes=5000,                # Number of episodes to train for\n",
  eval_frequency=500,               # Frequency of evaluations during training\n",
  eval_episodes=50,                 # Number of episodes for each evaluation\n",
  train_log_frequency=50,           # Frequency of logging during training\n",
  eval_log_frequency=1,             # Frequency of logging during evaluation\n",
 eval_render=False,                # Whether to render during evaluation\n",
 render=False,                     # Whether to render during training\n",
 attacker=True,                    # Whether the agent is an attacker\n",
defender=False,                   # Whether the agent is a defender\n",
 save_dir=result_dir,              # Directory to save results\n",
 save_frequency=500,               # Frequency to save the model during training\n",
 hidden_layers=[128, 128],         # Hidden layer sizes for the neural network\n",
 device=device                     # Device to run the neural network on\n",
    )
agent = DDQNAgent(env, ddqn_config)

# Print agent information
print(f"Agent state dimension: {agent.state_dim}")
print(f"Agent action dimension: {agent.num_actions}")
print(f"Neural network hidden layers: {ddqn_config.hidden_layers}")

Agent state dimension: 30
Agent action dimension: 30
Neural network hidden layers: [128, 128]


In [103]:
def print_summary(result, title=""):
    print(f"\n{title} Summary:")
    print("-"*50)
    print(f"- Average Reward: {np.mean(result.episode_rewards):.2f} ± {np.std(result.episode_rewards):.2f}")
    print(f"- Max Reward: {np.max(result.episode_rewards):.2f}")
    print(f"- Min Reward: {np.min(result.episode_rewards):.2f}")
    print(f"- Avg Episode Length: {np.mean(result.episode_steps):.2f} steps")

    # Call this after training
    print_summary(agent.train_result, "Training")
    print_summary(agent.eval_result, "Evaluation")

    print(f"- Cumulative Reward: {sum(result.episode_rewards)}")

def plot_training(agent):
    """Plot training progress"""
    plt.figure(figsize=(12,6))
    
    # Smoothed rewards
    rewards = agent.train_result.episode_rewards
    window_size = 50
    smooth_rewards = [np.mean(rewards[max(0,i-window_size):i+1]) 
                     for i in range(len(rewards))]
    
    plt.plot(smooth_rewards)
    plt.title("Training Progress (Smoothed Rewards)")
    plt.xlabel("Episode")
    plt.ylabel("Reward")
    plt.show()


In [104]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
import gym
import gym_idsgame
import time
import os
from datetime import datetime
from src.agents.ddqn_agent import DDQNConfig, DDQNAgent, create_ddqn_agent

# Set random seed
np.random.seed(42)
torch.manual_seed(42)

def extract_attacker_obs(obs):
    """Extract and flatten attacker observation"""
    if isinstance(obs, tuple):
        obs = obs[0]  # Handle gym>=0.26.0 reset() returning (obs, info)
    return np.array(obs).flatten()


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


# # Create environment
# env_name = "idsgame-minimal_defense-v2"
# env = gym.make(env_name)

# Create output directory
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
result_dir = f"./results/ddqn_{timestamp}"
if not os.path.exists(result_dir):
    os.makedirs(result_dir)


sample_obs = env.reset()[0] if isinstance(env.reset(), tuple) else env.reset()
state_dim = np.array(sample_obs).flatten().shape[0]
action_dim = env.attacker_action_space.n

print(f"Environment: {env_name}")
print(f"State dimension: {state_dim}")
print(f"Action dimension: {action_dim}")


# Create DDQN configuration
config = DDQNConfig(
    gamma=0.99,                       # Discount factor
    lr=1e-3,                          # Learning rate
    batch_size=32,                    # Reduce batch size to improve stability
    epsilon=1.0,                      # Initial exploration rate
    epsilon_decay=0.995,              # Decay rate for epsilon
    min_epsilon=0.01,                 # Minimum value for epsilon
    target_network_update_freq=10,    # Update target network frequency
    replay_memory_size=10000,         # Size of replay buffer
    num_episodes=10000,               # Number of episodes to train for
    train_log_frequency=100,          # Log progress every 100 episodes
    eval_frequency=1000,              # Evaluate every 1000 episodes
    save_dir=result_dir,              # Directory to save results
    hidden_layers=[64, 64]            # Smaller hidden layers for stability
)
agent = DDQNAgent(env, config)

# Training parameters
num_episodes = 20000
max_steps = 100
log_frequency = 500

# For tracking progress
reward_history = []
episode_lengths = []
start_time = time.time()

print(f"Starting training for {num_episodes} episodes...")

# Training loop
for ep in range(num_episodes):
    # Reset environment
    obs = env.reset()
    if isinstance(obs, tuple):
        obs = obs[0]
        
    state = extract_attacker_obs(obs)  # Make sure state is properly formatted
    
    total_reward = 0
    steps = 0
    done = False
    
    # Episode loop
    for t in range(max_steps):
        # Select attacker action
        action = agent.select_action(state)
        
        # Get defender action (random)
        defense_action = env.defender_action_space.sample()
        
        # Combine actions for the environment
        full_action = (action, defense_action)
        
        # Take step in environment
        next_obs, reward, terminated, truncated, info = env.step(full_action)
        done = terminated or truncated
        
        # Get attacker reward
        attacker_reward = reward[0] if isinstance(reward, tuple) else reward
        
        # Extract and format next state
        next_state = extract_attacker_obs(next_obs)
        
        # Store transition in replay buffer
        agent.replay_buffer.push(state, action, attacker_reward, next_state, bool(done))
        
        # Update state and metrics
        state = next_state
        total_reward += attacker_reward
        steps += 1
        
        if done:
            break
            
    # Store episode metrics
    reward_history.append(total_reward)
    episode_lengths.append(steps)
    
    # Log progress
    if (ep + 1) % log_frequency == 0 or ep == 0:
        avg_reward = np.mean(reward_history[-min(log_frequency, len(reward_history)):]) 
        elapsed_time = time.time() - start_time
        print(f"[Ep {ep+1}/{num_episodes}] AvgReward (last {min(log_frequency, len(reward_history))}): {avg_reward:.2f} | "
              f"Total: {total_reward:.2f} | Epsilon: {agent.epsilon:.3f} | "
              f"Steps: {steps} | Time: {elapsed_time:.1f}s")

# Agent training should have been updating automatically through the 
# agent's internal train() method, if implemented properly

training_time = time.time() - start_time
print(f"\nTraining completed in {training_time:.1f} seconds")

# Calculate evaluation metrics
result = {
    "average_reward": np.mean(reward_history),
    "reward_std": np.std(reward_history),
    "max_reward": np.max(reward_history),
    "min_reward": np.min(reward_history),
    "average_episode_length": np.mean(episode_lengths),
    "episode_length_std": np.std(episode_lengths),
    "max_episode_length": np.max(episode_lengths),
    "min_episode_length": np.min(episode_lengths),
    "average_hack_probability": 0.0,
    "hack_probability_std": 0.0,
    "max_hack_probability": 0.0,
    "min_hack_probability": 0.0,
    "cumulative_reward": int(np.sum(reward_history)),
}

# Print evaluation summary
print("\n📊 Final DDQN Training Performance:")
print(env_name)
print('Results: ', result)
print(f"- Average Reward: {result['average_reward']:.2f} ± {result['reward_std']:.2f}")
print(f"- Max-Min Reward: {result['max_reward']:.2f} - {result['min_reward']:.2f}")
print(f"- Average Episode Length: {result['average_episode_length']:.2f} ± {result['episode_length_std']:.2f}")
print(f"- Cumulative Reward: {result['cumulative_reward']}")

# Save final model if agent has save_model method
if hasattr(agent, 'save_model'):
    agent.save_model()

Using device: cpu
Environment: idsgame-random_attack-v8
State dimension: 33
Action dimension: 30
Starting training for 20000 episodes...
[Ep 1/20000] AvgReward (last 1): -1.00 | Total: -1.00 | Epsilon: 1.000 | Steps: 2 | Time: 0.0s
[Ep 500/20000] AvgReward (last 500): -0.12 | Total: -1.00 | Epsilon: 1.000 | Steps: 6 | Time: 0.7s
[Ep 1000/20000] AvgReward (last 500): 0.01 | Total: -1.00 | Epsilon: 1.000 | Steps: 3 | Time: 1.3s
[Ep 1500/20000] AvgReward (last 500): -0.08 | Total: -1.00 | Epsilon: 1.000 | Steps: 3 | Time: 1.8s
[Ep 2000/20000] AvgReward (last 500): -0.05 | Total: -1.00 | Epsilon: 1.000 | Steps: 4 | Time: 2.4s
[Ep 2500/20000] AvgReward (last 500): 0.03 | Total: -1.00 | Epsilon: 1.000 | Steps: 1 | Time: 3.0s
[Ep 3000/20000] AvgReward (last 500): -0.09 | Total: 0.20 | Epsilon: 1.000 | Steps: 9 | Time: 3.5s
[Ep 3500/20000] AvgReward (last 500): -0.06 | Total: 0.12 | Epsilon: 1.000 | Steps: 10 | Time: 4.1s
[Ep 4000/20000] AvgReward (last 500): -0.03 | Total: -1.00 | Epsilon: 1.

2025-04-11 04:14:25,809 - DDQN_Agent - INFO - Model saved to: ./results/ddqn_2025-04-11_04-14-02/model
2025-04-11 04:14:25,809 - DDQN_Agent - INFO - Model saved to: ./results/ddqn_2025-04-11_04-14-02/model
2025-04-11 04:14:25,809 - DDQN_Agent - INFO - Model saved to: ./results/ddqn_2025-04-11_04-14-02/model
2025-04-11 04:14:25,809 - DDQN_Agent - INFO - Model saved to: ./results/ddqn_2025-04-11_04-14-02/model
2025-04-11 04:14:25,809 - DDQN_Agent - INFO - Model saved to: ./results/ddqn_2025-04-11_04-14-02/model
2025-04-11 04:14:25,809 - DDQN_Agent - INFO - Model saved to: ./results/ddqn_2025-04-11_04-14-02/model
2025-04-11 04:14:25,809 - DDQN_Agent - INFO - Model saved to: ./results/ddqn_2025-04-11_04-14-02/model
2025-04-11 04:14:25,809 - DDQN_Agent - INFO - Model saved to: ./results/ddqn_2025-04-11_04-14-02/model
2025-04-11 04:14:25,809 - DDQN_Agent - INFO - Model saved to: ./results/ddqn_2025-04-11_04-14-02/model
2025-04-11 04:14:25,809 - DDQN_Agent - INFO - Model saved to: ./results/d

[Ep 20000/20000] AvgReward (last 500): -0.11 | Total: -1.00 | Epsilon: 1.000 | Steps: 1 | Time: 23.4s

Training completed in 23.4 seconds

📊 Final DDQN Training Performance:
idsgame-random_attack-v8
Results:  {'average_reward': -0.03574216806885925, 'reward_std': 1.461620471100011, 'max_reward': 4.0, 'min_reward': -1.0, 'average_episode_length': 4.6574, 'episode_length_std': 3.588735883288153, 'max_episode_length': 28, 'min_episode_length': 1, 'average_hack_probability': 0.0, 'hack_probability_std': 0.0, 'max_hack_probability': 0.0, 'min_hack_probability': 0.0, 'cumulative_reward': -714}
- Average Reward: -0.04 ± 1.46
- Max-Min Reward: 4.00 - -1.00
- Average Episode Length: 4.66 ± 3.59
- Cumulative Reward: -714


In [110]:
import os
import csv
import matplotlib.pyplot as plt


def save_rewards_to_csv(reward_history, filename="rewards.csv"):
    with open(filename, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Episode", "Reward"])
        for i, reward in enumerate(reward_history):
            writer.writerow([i, reward])

def plot_rewards(reward_history, title="Reward Over Episodes", save_path="DDQNreward_plot.png"):
    plt.figure(figsize=(10, 5))
    plt.plot(reward_history, label="Reward")
    plt.xlabel("Episode")
    plt.ylabel("Total Reward")
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.savefig(save_path)
    plt.close()

In [111]:
save_rewards_to_csv(reward_history, filename="DDQN_rewards.csv")
plot_rewards(reward_history, title="DDQN Training Rewards")